# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0634b18910>
├── 'a' --> tensor([[ 1.3722, -0.9412, -0.5144],
│                   [-0.8652,  1.1267,  0.7837]])
└── 'x' --> <FastTreeValue 0x7f0634b18b50>
    └── 'c' --> tensor([[-3.5516,  0.1473, -0.8741, -0.6011],
                        [-1.5128, -0.2074,  0.1758, -0.7763],
                        [ 0.4465, -1.9325,  0.7438,  0.8102]])

In [4]:
t.a

tensor([[ 1.3722, -0.9412, -0.5144],
        [-0.8652,  1.1267,  0.7837]])

In [5]:
%timeit t.a

66.7 ns ± 0.0226 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0634b18910>
├── 'a' --> tensor([[ 0.0510, -0.8554,  2.1770],
│                   [ 1.3586,  1.6602,  0.9174]])
└── 'x' --> <FastTreeValue 0x7f0634b18b50>
    └── 'c' --> tensor([[-3.5516,  0.1473, -0.8741, -0.6011],
                        [-1.5128, -0.2074,  0.1758, -0.7763],
                        [ 0.4465, -1.9325,  0.7438,  0.8102]])

In [7]:
%timeit t.a = new_value

69.2 ns ± 0.14 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.3722, -0.9412, -0.5144],
               [-0.8652,  1.1267,  0.7837]]),
    x: Batch(
           c: tensor([[-3.5516,  0.1473, -0.8741, -0.6011],
                      [-1.5128, -0.2074,  0.1758, -0.7763],
                      [ 0.4465, -1.9325,  0.7438,  0.8102]]),
       ),
)

In [10]:
b.a

tensor([[ 1.3722, -0.9412, -0.5144],
        [-0.8652,  1.1267,  0.7837]])

In [11]:
%timeit b.a

65 ns ± 0.0214 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.0683, -0.8154, -0.7523],
               [ 1.2928,  3.7835, -1.2237]]),
    x: Batch(
           c: tensor([[-3.5516,  0.1473, -0.8741, -0.6011],
                      [-1.5128, -0.2074,  0.1758, -0.7763],
                      [ 0.4465, -1.9325,  0.7438,  0.8102]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 0.128 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

847 ns ± 6.88 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 3.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 710 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 487 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f058a7c9ac0>
├── 'a' --> tensor([[[ 1.3722, -0.9412, -0.5144],
│                    [-0.8652,  1.1267,  0.7837]],
│           
│                   [[ 1.3722, -0.9412, -0.5144],
│                    [-0.8652,  1.1267,  0.7837]],
│           
│                   [[ 1.3722, -0.9412, -0.5144],
│                    [-0.8652,  1.1267,  0.7837]],
│           
│                   [[ 1.3722, -0.9412, -0.5144],
│                    [-0.8652,  1.1267,  0.7837]],
│           
│                   [[ 1.3722, -0.9412, -0.5144],
│                    [-0.8652,  1.1267,  0.7837]],
│           
│                   [[ 1.3722, -0.9412, -0.5144],
│                    [-0.8652,  1.1267,  0.7837]],
│           
│                   [[ 1.3722, -0.9412, -0.5144],
│                    [-0.8652,  1.1267,  0.7837]],
│           
│                   [[ 1.3722, -0.9412, -0.5144],
│                    [-0.8652,  1.1267,  0.7837]]])
└── 'x' --> <FastTreeValue 0x7f058a7c9430>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 340 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0634ae2490>
├── 'a' --> tensor([[ 1.3722, -0.9412, -0.5144],
│                   [-0.8652,  1.1267,  0.7837],
│                   [ 1.3722, -0.9412, -0.5144],
│                   [-0.8652,  1.1267,  0.7837],
│                   [ 1.3722, -0.9412, -0.5144],
│                   [-0.8652,  1.1267,  0.7837],
│                   [ 1.3722, -0.9412, -0.5144],
│                   [-0.8652,  1.1267,  0.7837],
│                   [ 1.3722, -0.9412, -0.5144],
│                   [-0.8652,  1.1267,  0.7837],
│                   [ 1.3722, -0.9412, -0.5144],
│                   [-0.8652,  1.1267,  0.7837],
│                   [ 1.3722, -0.9412, -0.5144],
│                   [-0.8652,  1.1267,  0.7837],
│                   [ 1.3722, -0.9412, -0.5144],
│                   [-0.8652,  1.1267,  0.7837]])
└── 'x' --> <FastTreeValue 0x7f0607f8d4f0>
    └── 'c' --> tensor([[-3.5516,  0.1473, -0.8741, -0.6011],
                        [-1.5128, -0.2074,  0.1758, -0.7763],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 79.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.3 µs ± 97.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-3.5516,  0.1473, -0.8741, -0.6011],
                       [-1.5128, -0.2074,  0.1758, -0.7763],
                       [ 0.4465, -1.9325,  0.7438,  0.8102]],
              
                      [[-3.5516,  0.1473, -0.8741, -0.6011],
                       [-1.5128, -0.2074,  0.1758, -0.7763],
                       [ 0.4465, -1.9325,  0.7438,  0.8102]],
              
                      [[-3.5516,  0.1473, -0.8741, -0.6011],
                       [-1.5128, -0.2074,  0.1758, -0.7763],
                       [ 0.4465, -1.9325,  0.7438,  0.8102]],
              
                      [[-3.5516,  0.1473, -0.8741, -0.6011],
                       [-1.5128, -0.2074,  0.1758, -0.7763],
                       [ 0.4465, -1.9325,  0.7438,  0.8102]],
              
                      [[-3.5516,  0.1473, -0.8741, -0.6011],
                       [-1.5128, -0.2074,  0.1758, -0.7763],
                       [ 0.4465, -1.9325,  0.7438,  0.8102]],

In [26]:
%timeit Batch.stack(batches)

78.9 µs ± 722 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-3.5516,  0.1473, -0.8741, -0.6011],
                      [-1.5128, -0.2074,  0.1758, -0.7763],
                      [ 0.4465, -1.9325,  0.7438,  0.8102],
                      [-3.5516,  0.1473, -0.8741, -0.6011],
                      [-1.5128, -0.2074,  0.1758, -0.7763],
                      [ 0.4465, -1.9325,  0.7438,  0.8102],
                      [-3.5516,  0.1473, -0.8741, -0.6011],
                      [-1.5128, -0.2074,  0.1758, -0.7763],
                      [ 0.4465, -1.9325,  0.7438,  0.8102],
                      [-3.5516,  0.1473, -0.8741, -0.6011],
                      [-1.5128, -0.2074,  0.1758, -0.7763],
                      [ 0.4465, -1.9325,  0.7438,  0.8102],
                      [-3.5516,  0.1473, -0.8741, -0.6011],
                      [-1.5128, -0.2074,  0.1758, -0.7763],
                      [ 0.4465, -1.9325,  0.7438,  0.8102],
                      [-3.5516,  0.1473, -0.8741, -0.6011],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 326 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

330 µs ± 2.24 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
